In [1]:
import pandas as pd
import json
import os 

In [2]:
directory = os.getcwd()

os.chdir(directory)

In [3]:
a = pd.read_csv(r"../../data/Cluster_data/HDB_Centroid_MRT pairing data_5_cluster.csv")
b = pd.read_csv(r"../../data/Cluster_data/Private_Centroid_MRT pairing data_5_cluster.csv")

with open(r'../../data/Cluster_data/Private_MRT_routes_5_cluster.json', 'r') as f:
    hdb_centroid_pair_route = json.load(f)

a['route'] = hdb_centroid_pair_route

with open(r'../../data/Cluster_data/HDB_MRT_routes_5_cluster.json', 'r') as f:
    private_centroid_pair_route = json.load(f)

b['route'] = private_centroid_pair_route

In [4]:
#Form a combined dataframe containing the closest 10 centroids for each MRT(5 from HDB and 5 from private)
c = pd.concat([a,b], axis = 0)

In [5]:
#Group by MRT and obtain the 5 closest centroids out of the 10 based on euclidean distance
d = c.groupby('MRT_Name').apply(lambda group: group.nsmallest(5, 'euclidean_distance'),include_groups = False).reset_index(drop=True)

In [6]:
#Reading in the MRT station data(Purely for left join to obtain the MRT station name subsequently)
mrt_stations_df = pd.read_csv(r"../../data/Cluster_data/mrt_station_final.csv",usecols = [1,2,3])
mrt_stations_df.sort_values(by='MRT.Name', inplace=True)
mrt_stations_df.reset_index(inplace=True)
mrt_stations_df.head()

,index,MRT.Name,Latitude,Longitude
0,12,ADMIRALTY MRT STATION,1.440589,103.800990
1,9,ALJUNIED MRT STATION,1.316433,103.882906
2,52,ANG MO KIO MRT STATION,1.369429,103.849455
3,115,BAKAU LRT STATION,1.387994,103.905415
4,54,BANGKIT LRT STATION,1.380022,103.772647


In [7]:
#Left join to obtain the MRT station name
d['MRT_Name'] = pd.merge(d, mrt_stations_df, left_on='index_y', right_on='index')['MRT.Name']

In [12]:
#Export the data to csv and json for use in visualization

# e = d.loc[:, d.columns != 'route']
# e.to_csv(r"../../data/Cluster_data/combined/combined_Centroid_MRT pairing data_5_cluster.csv")
# routes = d['route'].copy(deep = True)
# routes.to_json(r'../../data/Cluster_data/combined/combined_MRT_routes_5_cluster.json', orient='records')

In [8]:
#Group by MRT station and obtain the average distance of the 5 closest centroids

df = d.groupby('MRT_Name')['distance'].mean()
df = df.reset_index()
df

,MRT_Name,distance
0,ADMIRALTY MRT STATION,0.8634
1,ALJUNIED MRT STATION,0.7650
2,ANG MO KIO MRT STATION,1.7378
3,BAKAU LRT STATION,0.9404
4,BANGKIT LRT STATION,1.2698
...,...,...
170,WOODLANDS SOUTH MRT STATION,1.0780
171,WOODLEIGH MRT STATION,0.8986
172,YEW TEE MRT STATION,0.8758
173,YIO CHU KANG MRT STATION,1.5578


In [9]:
#sort the MRT stations by average distance
ranking = df.sort_values(by='distance')

In [14]:
#reset the index and add a rank column

ranking = ranking.reset_index(drop = True)
ranking['rank'] = ranking.index

ranking 

,MRT_Name,distance,rank
0,FARRER PARK MRT STATION,0.5734,0
1,PIONEER MRT STATION,0.6006,1
2,TIONG BAHRU MRT STATION,0.6754,2
3,RANGGUNG LRT STATION,0.7018,3
4,KEMBANGAN MRT STATION,0.7096,4
...,...,...,...
170,CORAL EDGE LRT STATION,6.6310,170
171,GUL CIRCLE MRT STATION,6.8662,171
172,TUAS CRESCENT MRT STATION,7.8248,172
173,TUAS WEST ROAD MRT STATION,9.5212,173


In [15]:
#ranking.to_csv(r"../../data/Cluster_data/combined/combined_MRT_ranking_5_cluster.csv")